In [ ]:
!pip install pycrfsutie
!pip install pyidaungsu

In [2]:
MMCONST = set(u'\u1000-\u1021')
MMVOWEL = set(u'\u1022-\u102A\u103F\u104C-\u104F')
MMNUMBER = set(u'၀-၉0-9')
MMSUB = set(u'្')
MMDIAC = set(u'\u102B-\u1038\u103A-\u103E')

EN = set(u'abcdefghijklmnopqrstuvwxyz0123456789')

NS = 'NS'
def get_type(chr):
  if chr.lower() in EN: return NS
  if chr in MMCONST: return "C"
  if chr in MMVOWEL: return "W"
  if chr in MMNUMBER: return NS
  if chr in MMSUB: return "S"
  if chr in MMDIAC: return "D"
  return NS

def is_no_space(k):
  if get_type(k[0])==NS: return True
  return False

def kcc_type(k):
  if len(k)==1: return get_type(k)
  else: return "K" + str(len(k))

def create_char_features(kccs, i):
    maxi = len(kccs)
    kcc = kccs[i]

    features = {
        'kcc': kcc,
        't': kcc_type(kcc),
        'ns': is_no_space(kcc)
    }
    if i >= 1:
        features.update({
            'kcc[-1]'  : kccs[i-1],
            'kcc[-1]t' : kcc_type(kccs[i-1]),
            'kcc[-1:0]': kccs[i-1] + kccs[i],
            'ns-1' : is_no_space(kccs[i-1])
        })
    else:
        features['BOS'] = True

    if i >= 2:
        features.update({
            'kcc[-2]'   : kccs[i-2],
            'kcc[-2]t'  : kcc_type(kccs[i-2]),
            'kcc[-2:-1]': kccs[i-2] + kccs[i-1],
            'kcc[-2:0]' : kccs[i-2] + kccs[i-1] + kccs[i],
        })
    if i >= 3:
        features.update({
            'kcc[-3]'   : kccs[i-3],
            'kcc[-3]t'  : kcc_type(kccs[i-3]),
            'kcc[-3:0]' : kccs[i-3] + kccs[i-2] + kccs[i-1] + kccs[i],
            'kcc[-3:-1]': kccs[i-3] + kccs[i-2] + kccs[i-1],
            'kcc[-3:-2]': kccs[i-3] + kccs[i-2],
        })

    if i < maxi-1:
        features.update({
            'kcc[+1]'  : kccs[i+1],
            'kcc[+1]t'  : kcc_type(kccs[i+1]),
            'kcc[+1:0]': kccs[i] + kccs[i+1],
            'ns+1' : is_no_space(kccs[i+1])

        })
    else:
        features['EOS'] = True

    if i < maxi-2:
        features.update({
            'kcc[+2]'   : kccs[i+2],
            'kcc[+2]t'   : kcc_type(kccs[i+2]),
            'kcc[+1:+2]': kccs[i+1] + kccs[i+2],
            'kcc[0:+2]' : kccs[i+0] + kccs[i+1] + kccs[i+2],
            'ns+2' : is_no_space(kccs[i+2])
        })
    if i < maxi-3:
        features.update({
            'kcc[+3]'   : kccs[i+3],
            'kcc[+3]t'   : kcc_type(kccs[i+3]),
            'kcc[+2:+3]': kccs[i+2] + kccs[i+3],
            'kcc[+1:+3]': kccs[i+1] + kccs[i+2] + kccs[i+3],
            'kcc[0:+3]' : kccs[i+0] + kccs[i+1] + kccs[i+2] + kccs[i+3],
        })
    return features


def create_sentence_features(kccs):
    return [create_char_features(kccs, i) for i in range(len(kccs))]

In [4]:
import pycrfsuite
import pyidaungsu as pds

sentence = "ယိုးဒယား အပ်မှို ထောင်း လုပ် စား ကြ မယ် ... လူငယ် တွေ ကြား မှာ အကြိုက် များ တဲ့ အပ်မှို ကို ဒီ တစ် ခေါက် မှာ ကင်မစား ပဲ ယိုးဒယားထောင်း လုပ် စား ကြည့် ရအောင် နော် ... လုပ် နည်း က အရမ်း မခက် ပဲ ဒီ နည်း အတိုင်း တခြား ကိုယ် ထောင်း ချင် တဲ့ သင်္ဘောသီး တို့ ၊ သခွားသီး တို့ ၊ ပြောင်းဖူး တို့ ကို လည်း ထောင်း စား လို့ ရ တယ် နော် ပါဝင် ပစ္စည်း များ - အပ်မှို တစ် ထုပ် (1350 ks ) - ခရမ်းချဉ်သီး အနေတော် တစ် လုံး ခွဲ ( ကြိုက် ရင် နှစ် လုံး ထည့် လို့ ရ ပါ တယ် ) - ထန်းလျက် ခဲ တစ် ခဲ ( အနေတော် လုံး ) - မြေပဲ ထမင်းစားဇွန်း တစ် ဇွန်း စာ ( ချိန်ဆ ပါ.မြေပဲ များ ရင် အီ ပြီး စားမကောင်း ပါ ) - ငရုတ်သီးစိမ်း ( အစပ် ကြိုက် ရင် အသေးလေး တွေ သုံး ပါ.အတောင့်ကြီး ၁၃ ~ ၁၅ တောင့် ) - သံပုရာသီး တစ် လုံး ( သံပုရာသီး မ ရှိ လို့ ရှောက်သီး ထည့် လိုက် ရ တယ် 😁 ) - ပုစွန်ခြောက် အနည်းငယ် ( အကောင်း ကြိုက် သူ များ လို သလောက် သာ ထည့် စား ) - ငံပြာရည် လက်ဖက် စား ဇွန်း ၂ ဇွန်း - အချိုမှုန့် အနည်းငယ် ပြင်ဆင် ပြုလုပ် နည်း အဆင့်ဆင့် - အပ်မှို ကို အမြစ်ဖြတ် ရေစင်စင်ဆေး ပြီး ရေနွေးဖျော ပါ ( ရေနွေး အဆင်သင့် မရှိ ရင် ပြုတ် ပါ . ၅ မိနစ် ) - ပြုတ် ထား တုန်း မြေပဲ , ထန်းလျက် , ငရုတ်သီးစိမ်း တို့ ကို ရောထောင်း ပါ. - ညက် သွား ရင် ခရမ်းချဉ်သီး လေး စိတ်လှီး ထည့် ပြီး ခဏ ထောင်း ပေး ပါ.- ရေနွေး ကြောင့် ပူ နေ တဲ့ အပ်မှို လေး တွေ ကို ရေအေး ခဏ စိမ် ပေး ပါ . ( သပ်သပ် မစိမ် ချင် လည်း အပူခံ နိုင် ရင် ရေနွေး ထဲ က ဆယ် ပါ . လှီး တဲ့ အခါ မပူ ရအောင် ရေအေး စိမ် ခိုင်း ခြင်း ပါ ) - အပ်မှို လေး များ ကို တစ်၀က် ဖြတ် ကာ အထက်ပါ ထောင်း ထား သည် များ နှင့် ရော , - ငံပြာရည် , အချိုမှုန့် , သံပုရာရည် ညှစ် ပြီး သုပ် ကာ ပုစွန်ခြောက် လေး ဖြူး ပြီး စားသုံး နိုင် ပါ ပြီ ရှင် အသားလုံး လေး တွေ နဲ့ တွဲ ပြီး သုပ်စား လည်း ရ တယ် နော် အရမ်း ကို အရသာရှိ ပြီး စား လို့ ကောင်း တဲ့ အထောင်း တစ် ခွက် မလို့ အိမ် မှာ လုပ် စား ကြည့် ကြ ဦး နော်"
context = sentence.split(" ")

tagger = pycrfsuite.Tagger()
tagger.open('/model/mm-data-pos-500-itr.crfsuite')

prediction = tagger.tag(create_sentence_features(context))

tagged_sentence = (list(zip(context, prediction)))
print(tagged_sentence)


context:  ['ယိုးဒယား', 'အပ်မှို', 'ထောင်း', 'လုပ်', 'စား', 'ကြ', 'မယ်', '...', 'လူငယ်', 'တွေ', 'ကြား', 'မှာ', 'အကြိုက်', 'များ', 'တဲ့', 'အပ်မှို', 'ကို', 'ဒီ', 'တစ်', 'ခေါက်', 'မှာ', 'ကင်မစား', 'ပဲ', 'ယိုးဒယားထောင်း', 'လုပ်', 'စား', 'ကြည့်', 'ရအောင်', 'နော်', '...', 'လုပ်', 'နည်း', 'က', 'အရမ်း', 'မခက်', 'ပဲ', 'ဒီ', 'နည်း', 'အတိုင်း', 'တခြား', 'ကိုယ်', 'ထောင်း', 'ချင်', 'တဲ့', 'သင်္ဘောသီး', 'တို့', '၊', 'သခွားသီး', 'တို့', '၊', 'ပြောင်းဖူး', 'တို့', 'ကို', 'လည်း', 'ထောင်း', 'စား', 'လို့', 'ရ', 'တယ်', 'နော်', 'ပါဝင်', 'ပစ္စည်း', 'များ', '-', 'အပ်မှို', 'တစ်', 'ထုပ်', '(1350', 'ks', ')', '-', 'ခရမ်းချဉ်သီး', 'အနေတော်', 'တစ်', 'လုံး', 'ခွဲ', '(', 'ကြိုက်', 'ရင်', 'နှစ်', 'လုံး', 'ထည့်', 'လို့', 'ရ', 'ပါ', 'တယ်', ')', '-', 'ထန်းလျက်', 'ခဲ', 'တစ်', 'ခဲ', '(', 'အနေတော်', 'လုံး', ')', '-', 'မြေပဲ', 'ထမင်းစားဇွန်း', 'တစ်', 'ဇွန်း', 'စာ', '(', 'ချိန်ဆ', 'ပါ.မြေပဲ', 'များ', 'ရင်', 'အီ', 'ပြီး', 'စားမကောင်း', 'ပါ', ')', '-', 'ငရုတ်သီးစိမ်း', '(', 'အစပ်', 'ကြိုက်', 'ရင်', 'အသေးလေး', 'တွေ', 'သုံး', 